In [13]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
from nltk import word_tokenize
import pickle

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

In [14]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
        

In [15]:
import codecs
short_pos = codecs.open("short_reviews/positive.txt","r",encoding="latin2").read()
short_neg = codecs.open("short_reviews/negative.txt","r", encoding="latin2").read()

In [34]:
all_words = []
documents = []

In [35]:
# to select only adjects = J, adverbs = R and verb = V we delted all the other words in the text.

allowed_word_types = ["J"]

for p in short_pos.split("\n"):
    documents.append((p,"pos"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

for p in short_neg.split("\n"):
    documents.append((p,"neg"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

In [36]:
documents[0][0]

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . '

In [37]:
for (rev, category) in documents:
    print(category)

pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos


pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos


pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos


neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg
neg


In [38]:
all_words = nltk.FreqDist(all_words)
print(all_words.most_common(100))
print(all_words["positive"])

[('good', 369), ('more', 331), ('little', 265), ('funny', 245), ('much', 234), ('bad', 234), ('best', 208), ('new', 206), ('own', 185), ('many', 183), ('most', 167), ('other', 167), ('great', 160), ('big', 156), ('few', 139), ('first', 133), ('real', 132), ('i', 122), ('better', 116), ('full', 116), ('such', 114), ('romantic', 112), ('american', 110), ('old', 106), ('same', 103), ('original', 100), ('human', 100), ('hard', 98), ('[', 97), ('interesting', 97), ('young', 93), ('enough', 92), ('emotional', 89), ('least', 87), ('long', 83), ('last', 82), ('cinematic', 82), ('true', 75), ('entertaining', 75), ('high', 71), ('special', 70), ('predictable', 70), (']', 69), ('visual', 69), ('familiar', 63), ('whole', 63), ('comic', 63), ('enjoyable', 62), ('sweet', 60), ('narrative', 60), ('less', 60), ('short', 59), ('worst', 59), ('strong', 58), ('only', 58), ('fascinating', 53), ('obvious', 52), ('solid', 52), ('powerful', 51), ('modern', 51), ('french', 51), ('rare', 50), ('fresh', 50), ('

In [39]:
#commonly used words to train against
word_features = list(all_words.keys())[:5000]

In [22]:
save_word_features = open("pickled_algorithms/word_features.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

In [40]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    
    return features

In [10]:
#print((find_features(movie_reviews.words("neg/cv000_29416.txt"))))

In [41]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

save_featuresets = open("pickled_algorithms/featuresets.pickle","wb")
pickle.dump(featuresets, save_featuresets)
save_featuresets.close()

In [42]:
random.shuffle(featuresets)

In [43]:
training_set = featuresets[:10000]
testing_set = featuresets[10000:]

In [44]:
# likelyhood = prior occurences x likihood / evidence

classifier = nltk.NaiveBayesClassifier.train(training_set)

#classifier_f = open("naivebayes.pickle", "rb")
#classifier = pickle.load(classifier_f)
#classifier_f.close()

print("Original Naive Bayes Algo accuracy:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

Original Naive Bayes Algo accuracy: 70.78313253012048
Most Informative Features
              engrossing = True              pos : neg    =     20.9 : 1.0
                    warm = True              pos : neg    =     18.3 : 1.0
                 generic = True              neg : pos    =     17.0 : 1.0
                mediocre = True              neg : pos    =     17.0 : 1.0
                 routine = True              neg : pos    =     15.7 : 1.0
               inventive = True              pos : neg    =     15.0 : 1.0
                    flat = True              neg : pos    =     14.6 : 1.0
                  boring = True              neg : pos    =     14.2 : 1.0
               wonderful = True              pos : neg    =     12.2 : 1.0
                mindless = True              neg : pos    =     11.0 : 1.0
                    dull = True              neg : pos    =     10.4 : 1.0
                  devoid = True              neg : pos    =     10.4 : 1.0
                   s

In [ ]:
# Mulitnominal classifier
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)
#MNB_classifier.show_most_informative_features(15)

In [ ]:
save_MNB_classifier = open("pickled_algorithms/MNB.pickle","wb")
pickle.dump(MNB_classifier, save_MNB_classifier)
save_MNB_classifier.close()

In [ ]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

In [ ]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

In [ ]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

In [ ]:
SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC_classifier accuracy:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

In [ ]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

In [ ]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

## Vote classifier

In [ ]:
voted_classifier = VoteClassifier(classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LinearSVC_classifier,
                                  NuSVC_classifier,
                                  SGDClassifier_classifier,
                                  LogisticRegression_classifier)

In [ ]:
print("voted_classifier accuracy:", (nltk.classify.accuracy(voted_classifier,testing_set))*100)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[0][0]), "Confidence %:",voted_classifier.confidence(testing_set[0][0])*100)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[1][0]), "Confidence %:",voted_classifier.confidence(testing_set[1][0])*100)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[2][0]), "Confidence %:",voted_classifier.confidence(testing_set[2][0])*100)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[3][0]), "Confidence %:",voted_classifier.confidence(testing_set[3][0])*100)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[4][0]), "Confidence %:",voted_classifier.confidence(testing_set[4][0])*100)

In [ ]:
print("Classification:", voted_classifier.classify(testing_set[5][0]), "Confidence %:",voted_classifier.confidence(testing_set[5][0])*100)

In [ ]:
def sentiment(text):
    feats = find_features(text)
    
    return voted_classifier.classify(feats)


## Pickling, how to

save_classifier = open("naivebayes.pickle", "wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()